In [1]:
# Import dependencies
import pandas as pd
import ast                          # importing abstract syntax tree to allow the parsing of string based lists as lists  
from sklearn.preprocessing import StandardScaler

### **Import and make a copy of CSV**

In [2]:
# Filepath
csv_filepath =  "../../../data/processed/cleaned_listings.csv"

# Read CSV
listings_csv = pd.read_csv(csv_filepath)

# make a copy
listings_df = listings_csv.copy()

In [3]:
listings_df

,id,latitude,longitude,neighbourhood,luxury_neighborhood_flag,room_type,property_type,standardized_property_type,luxury_score,accommodates,...,bathrooms,bathrooms_count,amenities,stay_duration,price,number_of_reviews_ly,host_is_superhost,host_listings_count,estimated_occupancy_l365d,estimated_revenue_l365d
0,696407278180533419,43.670920,-79.395190,Annex,0,Entire home/apt,Entire condo,apartment,0.191553,3,...,1 bath,1.0,"[""Hair dryer"", ""Central heating"", ""Paid parkin...",Mid-term,450.0,2,0,7,56,25200.0
1,696457318817239920,43.655704,-79.399910,Kensington-Chinatown,0,Private room,Private room in rental unit,budget,0.031755,2,...,1 private bath,1.0,"[""Hair dryer"", ""Central heating"", ""Shampoo"", ""...",Mid-term,78.0,2,1,8,112,8736.0
2,696602542310304703,43.648778,-79.401183,Kensington-Chinatown,0,Entire home/apt,Entire rental unit,apartment,0.070080,3,...,1 bath,1.0,"[""Hair dryer"", ""Room-darkening shades"", ""Keypa...",Short-term,132.0,4,1,1,224,29568.0
3,696973520016945803,43.612110,-79.539850,other,0,Entire home/apt,Entire rental unit,apartment,0.029755,2,...,1 bath,1.0,"[""Hair dryer"", ""Heating"", ""Shampoo"", ""Body soa...",Short-term,81.0,12,0,1,60,4860.0
4,697004718610327555,43.735735,-79.480703,other,0,Entire home/apt,Entire townhouse,home,0.104147,6,...,2 bath,2.5,"[""Room-darkening shades"", ""Keypad"", ""Elevator""...",Mid-term,236.0,15,0,1,90,21240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9621,1355385238383046824,43.645749,-79.393083,Waterfront Communities-The Island,1,Entire home/apt,Entire rental unit,apartment,0.064115,4,...,1 bath,1.0,"[""Hair dryer"", ""Elevator"", ""Central heating"", ...",Mid-term,170.0,0,1,1,24,4080.0
9622,1362331480938692703,43.649540,-79.388640,Waterfront Communities-The Island,1,Entire home/apt,Entire home,home,0.051650,6,...,1 bath,1.0,"[""Hair dryer"", ""Cleaning available during stay...",Mid-term,145.0,0,0,1,6,870.0
9623,1363197896119450651,43.653270,-79.396790,Kensington-Chinatown,0,Entire home/apt,Entire home,home,0.039307,2,...,1 bath,1.0,"[""Hair dryer"", ""Keypad"", ""Mini fridge"", ""Heati...",Mid-term,114.0,0,1,5,6,684.0
9624,1364280421231205552,43.705119,-79.272289,other,0,Private room,Private room in home,budget,0.015814,2,...,2 shared bath,2.0,"[""Hair dryer"", ""Room-darkening shades"", ""Firep...",Short-term,55.0,0,0,2,6,330.0


### **Feature Extraction - Categorical Columns Binary Encoding**

#### use pd.get_dummies on all categorical columns to binary encode for the ML Models

In [4]:
# Apply pd.get_dummies
listings_df = pd.get_dummies(listings_df, columns=['neighbourhood', 'room_type', 'standardized_property_type', 'stay_duration'], dtype=int)

### **Feature Extraction - Amenities**

#### Limit to Necessary Columns and apply Abstract Syntax Tree
Allows the stringified list in amenities to be turned into an accessible list

In [5]:
# Limit CSV to Amenities Column
listings_amenities = listings_df[['id', 'amenities']].copy()

# Apply AST
listings_amenities['amenities'] = listings_amenities['amenities'].apply(ast.literal_eval)

#### Explode Amenities

In [6]:
# Explode amenities Column
exploded_amenities = listings_amenities.explode('amenities')

#### Amenity Basic Cleanup

In [7]:
# Create Amenity Grouping Dictionary
amenity_grouping = {
    # Grouped Coffee makers
    "Coffee": "Coffee maker",
    "Coffee maker: drip coffee maker": "Coffee maker",
    "Coffee maker: Keurig coffee machine": "Coffee maker",

    # Heating / AC
    "Central heating": "Heating",
    "Central air conditioning": "Air conditioning",

    # Washers / Dryers
    "Free washer – In unit": "Washer",
    "Free dryer – In unit": "Dryer",

    # Cooking tools
    "Baking sheet": "Cooking basics",
    "Electric stove": "Stove",
    "Mini fridge": "Refrigerator",
    "Barbecue utensils": "BBQ grill",
    "Stainless steel oven": "Oven",

    # Privacy / security
    "Lock on bedroom door": "Private room lock",
    "Exterior security cameras on property": "Security camera",

    # Furniture
    "Clothing storage: closet": "Clothing storage",
    "Closet": "Clothing storage",
    "Clothing storage: closet and dresser": "Clothing storage",
    "Drying rack for clothing": "Drying rack",
    
    # Patios
    "Private patio or balcony": "Patio or balcony",
    
    # Extras
    "Exercise equipment": "Gym",
    "Shared gym in building": "Gym",
    "Private backyard – Fully fenced": "Backyard"
}

# Apply mapping to exploded_amenities - essentially renaming similar or same amenities
exploded_amenities['amenities'] = exploded_amenities['amenities'].replace(amenity_grouping)

#### Amenity Standardization (list)

In [8]:
# List of Amenities (to KEEP)
basic_amenities = [
    "Smoke alarm", "Carbon monoxide alarm", "Hot water", "Heating", "Essentials", 
    "Bed linens", "Shampoo", "Shower gel", "Body soap", "Hangers", "Hair dryer",
    "Conditioner", "Fire extinguisher", "Cleaning products", "Private room lock"
]

convenience_amenities = [
    "Wifi", "Kitchen", "Microwave", "Refrigerator", "Freezer", "Cooking basics",
    "Stove", "Oven", "Iron", "Air conditioning", "TV", "Dishwasher",
    "Coffee maker", "Hot water kettle", "Self check-in", "Elevator", "Drying rack",
    "Room-darkening shades", "Dishes and silverware", "Toaster", "Extra pillows and blankets",
    "First aid kit", "Wine glasses", "Lockbox", "Keypad", "Clothing storage",
    "Blender", "Portable fans", "Smart lock", "Rice maker", "Ethernet connection"
]

special_amenities = [
    "Hot tub", "Pool", "Private entrance", "BBQ grill", "Fire pit",
    "Outdoor dining area", "Patio or balcony", "Gym", "Lake access",
    "Indoor fireplace", "Sound system", "Game console", "EV charger",
    "Security camera", "Pets allowed", "Backyard", "City skyline view", "Board games"
]

# might remove these as they're administrative or potentially non relevant at best
other_amenities = [
    "Washer", "Dryer", "Bathtub", "Dining table", "Dedicated workspace",
    "Free parking on premises", "Paid parking on premises",
    "Free street parking", "Paid street parking off premises",
    "Books and reading material", "Outdoor furniture",
    "Long term stays allowed", "Luggage dropoff allowed", "Laundromat nearby", 
    "Host greets you", "Single level home", "Cleaning available during stay"
]

# put together for all amenities to be included
complete_amenities = special_amenities + other_amenities

#### Amenity Standardization

In [9]:
# Standardized Amenities
amenities_clean = []

# Cycle and filter out the amenities on the lists above
for index, row in exploded_amenities.iterrows():
    if row['amenities'] in complete_amenities:
        amenities_clean.append(row)

# Convert amenities_cleaned back into Dataframe
amenities_cleaned = pd.DataFrame(amenities_clean)

# Drop instances of duplicates
amenities_cleaned = amenities_cleaned.drop_duplicates()

#### Amenity Binary encoding by pd.get_dummies - **Method 1** for ML 
- may be best but also may just be considered to be a lot of noise ( will require a round of feature importance to determine whether all columns are beneficial)

In [10]:
# Use pd.get_dummies to make categories ingestible for ML
amenity_dummies = pd.get_dummies(amenities_cleaned, columns=['amenities'])

# Group by id and then sum rows to limit back to 1 row per listing
listing_binary_amenities = amenity_dummies.groupby('id').sum().reset_index()

# Clean up column titles
listing_binary_amenities.columns = listing_binary_amenities.columns.str.replace(" ", "_")

# Merge back into dataframe
listings_df = pd.merge(listings_df, listing_binary_amenities, on='id', how='inner')

#### Amenity Categorization

In [11]:
# Function to define amenity by predefined lists
def categorize_amenity(amenity):
    if amenity in basic_amenities:
        return 'basic'
    elif amenity in convenience_amenities:
        return 'convenience'
    elif amenity in special_amenities:
        return 'special'
    elif amenity in other_amenities :
        return 'other'

# Apply function to amenities and save to new column
amenities_cleaned['amenity_classification'] = amenities_cleaned['amenities'].apply(categorize_amenity)

#### Amenity Category counts by listing_id  - **Method 2** for ML
- Categorized by a system of 4 bins, though dictating via common sense might not be considered completely accurate.

In [12]:
# Count Categories
amenities_counts = amenities_cleaned.groupby(['id', 'amenity_classification']).size().unstack(fill_value=0).reset_index()

# Rename Columns
amenities_counts = amenities_counts.rename(columns={
    'basic': 'basic_amenities_count',
    'convenience': 'convenience_amenities_count',
    'special': 'special_amenities_count',
    'other': 'other_amenities_count'
})

# Total Amenity count
# amenities_counts['total_amenities_count'] = amenities_counts['basic_amenities_count'] + amenities_counts['convenience_amenities_count'] + amenities_counts['special_amenities_count'] + amenities_counts['other_amenities_count']

# Merge back into dataframe
# listings_df = pd.merge(listings_df, amenities_counts, on='id', how='inner')

### **Drop Select Features**
- Drop property_type as we have standardized_property_type.
- Drop luxury markers as they're derived from our target columns price and estimated revenue
- Drop bathrooms as we have bathrooms_count.
- Drop Room Type as we hav ea standardized and more descriptive Property Type
- Drop original amenities as we have one hot encoded it
    - Drop basic_amenities_count, convenience_amenities_count, other_amenities_count, special_amenities_count and total_amenities_count as well as we are using Amenity Selection Method 1 (look above)


In [13]:
# Drop ID
listings_price_features = listings_df.drop(columns=[
    'property_type', 'luxury_neighborhood_flag', 'luxury_score', 'bathrooms', 'amenities']) 

### **Standard Scaler**

- This will Scale the numerical columns that aren't hot encoded/ represent magnitude rather than those that represent membership (boolean)

- Not all models require to be scaled
    - Linear Regression, kmeans and nn Models definitely benefit
    - Decision Trees and Random Forest Models less so

In [14]:
# Should we scale for this next model? 
scale_data_boolean = True

In [15]:
listings_price_features.columns[0:15]

Index(['id', 'latitude', 'longitude', 'accommodates', 'bedrooms', 'beds',
       'bathrooms_count', 'price', 'number_of_reviews_ly', 'host_is_superhost',
       'host_listings_count', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'neighbourhood_Annex',
       'neighbourhood_Bay Street Corridor'],
      dtype='object')

In [16]:
# Columns to Scale
price_columns = [
    'latitude', 'longitude', 'accommodates', 'bedrooms', 'beds', 
    'bathrooms_count', 'number_of_reviews_ly', 'host_listings_count'
    ]
occupancy_columns = price_columns + ['price']

# make a copy of output dataframe
listings_occupancy_features = listings_price_features.copy()

# Create StandardScaler()
scaler = StandardScaler()

# Scale Numerical Columns
if scale_data_boolean:
    scaled_occupancy = scaler.fit_transform(listings_price_features[occupancy_columns])
    scaled_price = scaler.fit_transform(listings_occupancy_features[price_columns])
    
    scaled_occupancy_df = pd.DataFrame(scaled_occupancy, columns=occupancy_columns)
    scaled_price_df = pd.DataFrame(scaled_price, columns=price_columns)
    
    # Drop unscaled numerical columns and replace with scaled
    for col in occupancy_columns:
        listings_occupancy_features[col] = scaled_occupancy_df[col]
    
    for col in price_columns:
        listings_price_features[col] = scaled_price_df[col]

### **Check and Save Cleaned Data**

- keeping id, latitude and longitude for later visualization and demo stages

In [17]:
# Check Output Data
display(listings_occupancy_features)
display(listings_price_features)

,id,latitude,longitude,accommodates,bedrooms,beds,bathrooms_count,price,number_of_reviews_ly,host_is_superhost,...,amenities_Paid_parking_on_premises,amenities_Paid_street_parking_off_premises,amenities_Patio_or_balcony,amenities_Pets_allowed,amenities_Pool,amenities_Private_entrance,amenities_Security_camera,amenities_Single_level_home,amenities_Sound_system,amenities_Washer
0,696407278180533419,-0.250004,0.024253,-0.161239,-0.492537,-0.698373,-0.474890,2.144775,-0.688757,0,...,0,0,0,0,0,0,0,0,0,1
1,696457318817239920,-0.555148,-0.040161,-0.645561,-0.492537,-0.698373,-0.474890,-0.625100,-0.688757,1,...,0,0,0,0,0,0,0,0,0,0
2,696602542310304703,-0.694044,-0.057544,-0.161239,-0.492537,-0.698373,-0.474890,-0.223022,-0.575721,1,...,0,1,0,1,0,1,0,0,1,1
3,696973520016945803,-1.429381,-1.950083,-0.645561,-0.492537,0.178501,-0.474890,-0.602763,-0.123575,0,...,0,0,0,0,0,1,0,1,0,0
4,697004718610327555,1.049792,-1.142835,1.291729,0.618537,1.055375,2.081840,0.551352,0.045979,0,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,1355385238383046824,-0.754792,0.053015,0.323084,-0.492537,0.178501,-0.474890,0.059923,-0.801794,1,...,0,0,1,0,0,1,0,1,0,1
9590,1362331480938692703,-0.678759,0.113648,1.291729,0.618537,1.055375,-0.474890,-0.126225,-0.801794,0,...,0,0,1,1,0,1,0,0,0,1
9591,1363197896119450651,-0.603958,0.002416,-0.645561,-0.492537,-0.698373,-0.474890,-0.357048,-0.801794,1,...,0,0,0,0,0,0,1,0,0,1
9592,1364280421231205552,0.435821,1.701619,-0.645561,-0.492537,-0.698373,1.229597,-0.796356,-0.801794,0,...,0,0,0,0,0,0,1,0,0,0


,id,latitude,longitude,accommodates,bedrooms,beds,bathrooms_count,price,number_of_reviews_ly,host_is_superhost,...,amenities_Paid_parking_on_premises,amenities_Paid_street_parking_off_premises,amenities_Patio_or_balcony,amenities_Pets_allowed,amenities_Pool,amenities_Private_entrance,amenities_Security_camera,amenities_Single_level_home,amenities_Sound_system,amenities_Washer
0,696407278180533419,-0.250004,0.024253,-0.161239,-0.492537,-0.698373,-0.474890,450.0,-0.688757,0,...,0,0,0,0,0,0,0,0,0,1
1,696457318817239920,-0.555148,-0.040161,-0.645561,-0.492537,-0.698373,-0.474890,78.0,-0.688757,1,...,0,0,0,0,0,0,0,0,0,0
2,696602542310304703,-0.694044,-0.057544,-0.161239,-0.492537,-0.698373,-0.474890,132.0,-0.575721,1,...,0,1,0,1,0,1,0,0,1,1
3,696973520016945803,-1.429381,-1.950083,-0.645561,-0.492537,0.178501,-0.474890,81.0,-0.123575,0,...,0,0,0,0,0,1,0,1,0,0
4,697004718610327555,1.049792,-1.142835,1.291729,0.618537,1.055375,2.081840,236.0,0.045979,0,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,1355385238383046824,-0.754792,0.053015,0.323084,-0.492537,0.178501,-0.474890,170.0,-0.801794,1,...,0,0,1,0,0,1,0,1,0,1
9590,1362331480938692703,-0.678759,0.113648,1.291729,0.618537,1.055375,-0.474890,145.0,-0.801794,0,...,0,0,1,1,0,1,0,0,0,1
9591,1363197896119450651,-0.603958,0.002416,-0.645561,-0.492537,-0.698373,-0.474890,114.0,-0.801794,1,...,0,0,0,0,0,0,1,0,0,1
9592,1364280421231205552,0.435821,1.701619,-0.645561,-0.492537,-0.698373,1.229597,55.0,-0.801794,0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
# Save Cleaned Data as CSV
if scale_data_boolean:
    listings_occupancy_features.to_csv('../../../data/processed/listings_feature_matrix_scaled_occupancy.csv', index=False)
    listings_price_features.to_csv('../../../data/processed/listings_feature_matrix_scaled_price.csv', index=False)
else:
    listings_price_features.to_csv('../../../data/processed/listings_feature_matrix.csv', index=False)
